# Setup Gym Environment

```py
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05 # when to give up
my_config["start_obs_capture"] = 0.05 # when to capture 
my_config["time_step_timeout_factor"] = 1.0 # how late is OK
my_config["act_buf_len"] = 3 # how many past actions
my_config["reset_act_buf"] = True # resect action buffer on reset
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2
```

This section needs to be setup for any method

In [1]:
debugAsGym = False

from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 312
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'img_hist_len': 3,
  'modelMode': 3,
  'agent' : 'A3C',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 2, # 1 is High Speed Ring, 2 is 0-400m, 
}

In [2]:
if debugAsGym:
    env = gymnasium.make("real-time-gym-v1", config=my_config)

In [3]:
if debugAsGym:
    obs,info = env.reset()

In [4]:
if debugAsGym:
    obs, info = env.reset()
    env.render()
    print("Obs_actual (shape)")
    [print(o.shape) for o in obs]

In [5]:
if debugAsGym:
    print("Obs_space (shape)")
    [print(o.shape) for o in env.observation_space]

In [6]:
if debugAsGym:
    import pprint
    obs, info = env.reset()
    obs, rew, terminated, truncated, info = env.step(env.action_space.sample())
    obs_space = env.observation_space
    while not (terminated or truncated):
        act = env.action_space.sample()
        obs, rew, terminated, truncated, info = env.step(act)
        env.render()


# Register the environment in a way that RLlib is happy

In [7]:
if not debugAsGym:
    def env_creator(env_config):
        env = gymnasium.make("real-time-gym-v1", config=env_config)
        return env  # return an env instance

    from ray.tune.registry import register_env
    register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [8]:
if not debugAsGym:
    from ray.rllib.algorithms.a3c import A3CConfig

    algo = (
        A3CConfig()
        .resources(
            num_gpus=1
            )
        .rollouts(
            num_rollout_workers=1,
            enable_connectors=True,
            batch_mode="complete_episodes",
            )
        .framework("torch")
        .environment(
            env="gt-rtgym-env-v1",
            disable_env_checking=True,
            render_env=False,
            )
        .training(
            #lr=tune.grid_search([0.01, 0.001, 0.0001])
            #lambda_=0.95,
            #gamma=0.99,
            #sgd_minibatch_size=128,
            train_batch_size=512,
            #num_sgd_iter=8,
            #clip_param=0.2,
            model={"fcnet_hiddens": [16, 16]},
        )
        .build()
    )

2023-05-16 00:43:09,409	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(RolloutWorker pid=12500) GT Real Time instantiated
(RolloutWorker pid=12500) GT AI Server instantiated for rtgym
(RolloutWorker pid=12500) simple reward system
(RolloutWorker pid=12500) starting up on localhost port 9999
(RolloutWorker pid=12500) Waiting for a connection
(RolloutWorker pid=12500) Connection from ('127.0.0.1', 61728)
(RolloutWorker pid=12500) reset triggered
(RolloutWorker pid=12500) reload save for track : 3


2023-05-16 00:43:25,299	INFO trainable.py:172 -- Trainable.setup took 18.544 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [9]:
if not debugAsGym:
    result = algo.train()

(RolloutWorker pid=12500) reset triggered
(RolloutWorker pid=12500) reload save for track : 3


(RolloutWorker pid=12500) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.15394309999964406
(RolloutWorker pid=12500)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=12500) reset triggered
(RolloutWorker pid=12500) reload save for track : 3


(RolloutWorker pid=12500) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.14840790000025095
(RolloutWorker pid=12500)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


In [10]:
algo.save()

'C:\\Users\\nadir/ray_results\\A3C_gt-rtgym-env-v1_2023-05-16_00-43-06lvgo_2s3\\checkpoint_000001'

In [11]:
if not debugAsGym:
    N = 2000

    for n in range(N):
        result = algo.train()
        print("Loop: ", n)
        if n % 20 == 0:
            print("Saved", n)
            algo.save()
    algo.save()

(RolloutWorker pid=12500) reset triggered
(RolloutWorker pid=12500) reload save for track : 3


(RolloutWorker pid=12500) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.16112930000177528
(RolloutWorker pid=12500)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=12500) reset triggered
(RolloutWorker pid=12500) reload save for track : 3


(RolloutWorker pid=12500) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.13678250000177172
(RolloutWorker pid=12500)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  0
Saved 0
(RolloutWorker pid=12500) reset triggered
(RolloutWorker pid=12500) reload save for track : 3


...

(RolloutWorker pid=12500) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.14849359988602373
(RolloutWorker pid=12500)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  93


2023-05-16 02:22:52,686	ERROR actor_manager.py:507 -- Ray error, taking actor 1 out of service. ray::RolloutWorker.apply() (pid=12500, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x000001D603F55D00>)
  File "python\ray\_raylet.pyx", line 877, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\_private\function_manager.py", line 670, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
    return method(self, *_args, **_kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\rllib\utils\actor_manager.py", line 185, in apply
    raise e
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-pack

RayTaskError(ValueError): [36mray::RolloutWorker.apply()[39m (pid=12500, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x000001D603F55D00>)
  File "python\ray\_raylet.pyx", line 877, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\_private\function_manager.py", line 670, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
    return method(self, *_args, **_kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\rllib\utils\actor_manager.py", line 185, in apply
    raise e
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\rllib\utils\actor_manager.py", line 176, in apply
 

2023-05-16 02:22:58,084	ERROR worker.py:408 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::RolloutWorker.apply() (pid=12500, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x000001D603F55D00>)
  File "python\ray\_raylet.pyx", line 877, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\_private\function_manager.py", line 670, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
    return method(self, *_args, **_kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\rllib\utils\actor_manager.py", line 185, in apply
    raise e
  File "c:\Users\nadir\anaconda3\envs\GTA

In [1]:
debugAsGym = False

from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 312
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'img_hist_len': 3,
  'modelMode': 3,
  'agent' : 'A3C',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 2, # 1 is High Speed Ring, 2 is 0-400m, 
}

if not debugAsGym:
    def env_creator(env_config):
        env = gymnasium.make("real-time-gym-v1", config=env_config)
        return env  # return an env instance

    from ray.tune.registry import register_env
    register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

from ray.rllib.algorithms.algorithm import Algorithm
algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/A3C_gt-rtgym-env-v1_2023-05-16_00-43-06lvgo_2s3/checkpoint_000082")

2023-05-16 07:25:32,316	WARNING checkpoints.py:109 -- No `rllib_checkpoint.json` file found in checkpoint directory C:/Users/nadir/ray_results/A3C_gt-rtgym-env-v1_2023-05-16_00-43-06lvgo_2s3/checkpoint_000082! Trying to extract checkpoint info from other files found in that dir.
2023-05-16 07:25:34,916	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(RolloutWorker pid=8384) GT Real Time instantiated
(RolloutWorker pid=8384) GT AI Server instantiated for rtgym
(RolloutWorker pid=8384) simple reward system
(RolloutWorker pid=8384) starting up on localhost port 9999
(RolloutWorker pid=8384) Waiting for a connection
(RolloutWorker pid=8384) Connection from ('127.0.0.1', 50589)
(RolloutWorker pid=8384) reset triggered
(RolloutWorker pid=8384) reload save for track : 3


2023-05-16 07:25:52,179	INFO trainable.py:172 -- Trainable.setup took 19.824 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [2]:
algo.train()

(RolloutWorker pid=8384) Exception in thread Thread-3:
(RolloutWorker pid=8384) Traceback (most recent call last):
(RolloutWorker pid=8384)   File "python\ray\_raylet.pyx", line 921, in ray._raylet.execute_task
(RolloutWorker pid=8384)   File "python\ray\_raylet.pyx", line 877, in ray._raylet.execute_task
(RolloutWorker pid=8384)   File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task
(RolloutWorker pid=8384)   File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor
(RolloutWorker pid=8384)   File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\_private\function_manager.py", line 670, in actor_method_executor
(RolloutWorker pid=8384)     return method(__ray_actor, *args, **kwargs)
(RolloutWorker pid=8384)   File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
(RolloutWorker pid=8384)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=8384)   File "c:

{'custom_metrics': {},
 'episode_media': {},
 'info': {'learner': {},
  'num_env_steps_sampled': 50544,
  'num_env_steps_trained': 50544,
  'num_agent_steps_sampled': 50544,
  'num_agent_steps_trained': 50544},
 'sampler_results': {'episode_reward_max': nan,
  'episode_reward_min': nan,
  'episode_reward_mean': nan,
  'episode_len_mean': nan,
  'episode_media': {},
  'episodes_this_iter': 0,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mean': {},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [], 'episode_lengths': []},
  'sampler_perf': {},
  'num_faulty_episodes': 0,
  'connector_metrics': {}},
 'episode_reward_max': nan,
 'episode_reward_min': nan,
 'episode_reward_mean': nan,
 'episode_len_mean': nan,
 'episodes_this_iter': 0,
 'policy_reward_min': {},
 'policy_reward_max': {},
 'policy_reward_mean': {},
 'hist_stats': {'episode_reward': [], 'episode_lengths': []},
 'sampler_perf': {},
 'num_faulty_episodes': 0,
 'connector_metrics': {},
 'num_